In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/MACROBATT/FINAL_SURVIVOR_DEATH/ZEROSHOT2/dataset.csv")
df.head()

,Filename,Text,Target
0,28353604_df.csv,A 69-year-old man became aware of the onset of...,3
1,28559815_df.csv,"A 34-year-old woman, 1 week postpartum, presen...",0
2,28538413_df.csv,A 63-year-old male patient without smoking or ...,2
3,28353588_df.csv,A 75-year-old man was referred to our hospital...,0
4,28353596_df.csv,A 47-year-old female patient presented progres...,2


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", device=0) #GPU

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model='cross-encoder/nli-roberta-base')


In [5]:
labels_dict = {'NEUTRAL':0,
                'LOW':1,
                'MODERATE':2,
                'HIGH':3}
def get_key(value,labels_dict):
  for key in labels_dict:
    if value == labels_dict[key]:
      return key
  return False


In [6]:
from tqdm import tqdm
final_list = []

candidate_labels = ["survivor", "death"]
for index in tqdm(range(len(df)),total=len(df)):
  text = df.iloc[index]["Text"]
  filename = df.iloc[index]["Filename"]
  results = classifier(text, candidate_labels)
  prediction = results['labels'][0]
  actual_label = get_key(df.iloc[index]["Target"],labels_dict)

  # print(filename,prediction,actual_label)
  final_list.append([filename,text,actual_label,prediction])


100%|██████████| 200/200 [10:26<00:00,  3.13s/it]


In [7]:
final_df = pd.DataFrame(final_list,columns=['Filename','Text','API KEY Prediction','Predicted'])
final_df

,Filename,Text,API KEY Prediction,Predicted
0,28353604_df.csv,A 69-year-old man became aware of the onset of...,HIGH,survivor
1,28559815_df.csv,"A 34-year-old woman, 1 week postpartum, presen...",NEUTRAL,survivor
2,28538413_df.csv,A 63-year-old male patient without smoking or ...,MODERATE,survivor
3,28353588_df.csv,A 75-year-old man was referred to our hospital...,NEUTRAL,survivor
4,28353596_df.csv,A 47-year-old female patient presented progres...,MODERATE,survivor
...,...,...,...,...
195,21477357_df.csv,A 52-year-old man (body surface area: 1.3 m2) ...,HIGH,survivor
196,18258107_df.csv,"Here, we describe another case in a 60-year-ol...",LOW,survivor
197,19860007_df.csv,A 70-year-old man was referred to our hospital...,HIGH,survivor
198,18236639_df.csv,A 30-year-old female (65 kg) underwent rhinopl...,MODERATE,survivor


In [8]:
final_df.to_csv("/content/drive/MyDrive/MACROBATT/FINAL_SURVIVOR_DEATH/ZEROSHOT2/pretrained_zeroshot-results.csv",index=False)

## Prediction of Death vs Survivour Dataset

In [9]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/MACROBATT/FINAL_SURVIVOR_DEATH/ZEROSHOT2/pretrained_zero_shot2_predicted_actual.csv")
df.head()

,Filename,Text,API KEY Prediction,Predicted,ACTUAL
0,15939911_df.csv,CASE: A 28-year-old previously healthy man pre...,NEUTRAL,survivor,survivor
1,16778410_df.csv,The patient was a 34-yr-old man who presented ...,HIGH,survivor,survivor
2,17803823_df.csv,A 23 year old white male with a 4 year history...,LOW,survivor,survivor
3,18236639_df.csv,A 30-year-old female (65 kg) underwent rhinopl...,MODERATE,survivor,survivor
4,18258107_df.csv,"Here, we describe another case in a 60-year-ol...",LOW,survivor,survivor


In [10]:
from tqdm import tqdm
count_true = 0
true_positive = 0 # case predicted as survivour and in actual the case is survivour
false_negative = 0 # case predicted as survivour and in actual the case is dead
true_negative = 0 # case predicted as death and in actual the case is death
false_positive = 0 # case predicted as death and in actual the case is survivour

for index in tqdm(range(len(df[:200]))):
  if df.iloc[index].Predicted==df.iloc[index].ACTUAL:
    count_true +=1

  if df.iloc[index].Predicted=="survivor" and df.iloc[index].ACTUAL=="survivor":
    true_positive +=1
  elif df.iloc[index].Predicted=="survivor" and df.iloc[index].ACTUAL=="death":
    false_negative +=1
  elif df.iloc[index].Predicted=="death" and df.iloc[index].ACTUAL=="death":
    true_negative +=1
  elif df.iloc[index].Predicted=="death" and df.iloc[index].ACTUAL=="survivor":
    false_positive +=1







100%|██████████| 200/200 [00:00<00:00, 5318.57it/s]


In [11]:
# survivor
Precision = true_positive / (true_positive + false_positive)
Recall = true_positive / (true_positive + false_negative)
F1_Score = 2 * ((Precision*Recall)/(Precision+Recall))

In [12]:
print("Accuracy: ",count_true/len(df[:200]))
print("Precision: ",Precision)
print("Recall: ",Recall)
print("F1 Score: ",F1_Score)

Accuracy:  0.855
Precision:  0.9940828402366864
Recall:  0.8571428571428571
F1 Score:  0.9205479452054794


In [13]:
#death
Precision = true_negative/ (true_negative + false_negative)
Recall = true_negative / (true_negative + false_positive)
F1_Score = 2 * ((Precision*Recall)/(Precision+Recall))

In [14]:
print("Accuracy: ",count_true/len(df[:200]))
print("Precision: ",Precision)
print("Recall: ",Recall)
print("F1 Score: ",F1_Score)

Accuracy:  0.855
Precision:  0.0967741935483871
Recall:  0.75
F1 Score:  0.1714285714285714
